In [2]:
import math
import os
import tempfile
import pandas as pd
import numpy as np
import shutil


In [8]:
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

os.chmod("/root/.kaggle/kaggle.json", 600)  # Atur izin file

In [9]:
!kaggle competitions download -c gammafest25

In [10]:
import zipfile
with zipfile.ZipFile("/content/gammafest25.zip", "r") as zip_ref:
    zip_ref.extractall("data")

In [3]:
df = pd.read_csv("/content/metadata.csv", encoding='latin-1') # Try 'latin-1' encoding, or others like 'cp1252', 'iso-8859-1' if 'latin-1' fails.
                                                                # Experiment with different encodings until you find the one that works.

In [4]:
df.drop(columns=['publication_date'], inplace=True)

In [5]:
import re

def clean_authors(authors):
    if pd.isna(authors):
        return ""
    # Ganti tanda ; menjadi koma
    authors = authors.replace(';', ',')
    # Ganti karakter non-alfabet/non-koma (selain koma dan spasi) jadi spasi
    authors = re.sub(r"[^a-zA-Z0-9,\s]", " ", authors)
    # Hapus spasi ganda
    authors = re.sub(r"\s+", " ", authors).strip()
    return authors


In [6]:
df['authors'] = df['authors'].apply(clean_authors)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4354 entries, 0 to 4353
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          4354 non-null   object
 1   title             4354 non-null   object
 2   publication_year  4354 non-null   int64 
 3   cited_by_count    4354 non-null   int64 
 4   type              4354 non-null   object
 5   authors           4354 non-null   object
 6   concepts          4354 non-null   object
dtypes: int64(2), object(5)
memory usage: 238.2+ KB


In [13]:
train_data = pd.read_csv('/content/data/train.csv')
merged_df = train_data.merge(df, on='paper_id', how='left')


KeyError: 'paper_id'

In [14]:
train_data = pd.read_csv('/content/data/train.csv')
# Assuming 'df' is the DataFrame read from 'metadata.csv', ensure 'paper_id' column exists
print(df.columns)  # Print columns of 'df' to verify if 'paper_id' exists
merged_df = train_data.merge(df, on='paper_id', how='left')

Index(['paper_id', 'title', 'publication_year', 'cited_by_count', 'type',
       'authors', 'concepts'],
      dtype='object')


KeyError: 'paper_id'

In [21]:
# Gabungkan metadata untuk kolom 'paper' pada train_data dengan 'paper_id' di df
merged_df = train_data.merge(df, left_on='paper', right_on='paper_id', how='left')

# Gabungkan metadata untuk kolom 'referenced_paper' pada train_data dengan 'paper_id' di df
merged_df = merged_df.merge(df, left_on='referenced_paper', right_on='paper_id', how='left', suffixes=('_paper', '_referenced'))

# Hasil dataset gabungan
# print(merged_df.head())


In [24]:
# Membuat fitur baru: selisih tahun publikasi
merged_df['year_difference'] = merged_df['publication_year_paper'] - merged_df['publication_year_referenced']

# Menghapus kolom publication_year_paper dan publication_year_referenced
merged_df.drop(columns=['publication_year_paper', 'publication_year_referenced'], inplace=True)

# Menampilkan beberapa baris data untuk memverifikasi hasil
merged_df.head()


,paper,referenced_paper,is_referenced,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_referenced,title_referenced,cited_by_count_referenced,type_referenced,authors_referenced,concepts_referenced,year_difference
0,p2128,p3728,0,p2128,A Survey of Data Augmentation Approaches for NLP,357,article,"Steven Y Feng, Varun Gangal, Jason Wei, Sarath...",Computer science; Popularity; Artificial intel...,p3728,Optimization Methods for Large-Scale Machine L...,2492,article,"L on Bottou, Frank E Curtis, Jorge Nocedal",Computer science; Machine learning; Artificial...,3
1,p0389,p3811,0,p0389,Residual Algorithms: Reinforcement Learning wi...,981,book-chapter,Leemon C Baird,Residual; Algorithm; Reinforcement learning; C...,p3811,Filter Pruning via Geometric Median for Deep C...,1078,article,"Yang He, Ping Liu, Ziwei Wang, Zhilan Hu, Yi Yang",FLOPS; Computer science; Convolutional neural ...,-24
2,p1298,p3760,0,p1298,SegNet: A Deep Convolutional Encoder-Decoder A...,16255,article,"Vijay Badrinarayanan, A C Kendall, Roberto Cip...",Computer science; Artificial intelligence; Ups...,p3760,Integrative methods for analyzing big data in ...,182,review,"Vladimir Gligorijevi , No l Malod Dognin, Nata...",Big data; Data science; Precision medicine; Re...,2
3,p0211,p1808,0,p0211,DeepLab: Semantic Image Segmentation with Deep...,18641,article,"Liang Chieh Chen, George Papandreou, Iasonas K...",Conditional random field; Artificial intellige...,p1808,Building Watson: An Overview of the DeepQA Pro...,1479,article,"David Ferrucci, Eric W Brown, Jennifer Chu Car...",Watson; Champion; IBM; Computer science; Archi...,7
4,p0843,p2964,0,p0843,Particle Swarm Optimization Algorithm and Its ...,799,review,Ahmed G Gad,Particle swarm optimization; Swarm intelligenc...,p2964,Linear Least-Squares algorithms for temporal d...,645,article,"Steven J Bradtke, Andrew G Barto",Recursive least squares filter; Algorithm; Tem...,26


In [27]:
merged_df.to_csv('merged_data.csv', index=False)

In [3]:
merged_df = pd.read_csv('merged_data.csv')

In [4]:
# Mengubah nilai year_difference menjadi -1 jika nilainya kurang dari 0
merged_df['year_difference'] = merged_df['year_difference'].apply(lambda x: -1 if x < 0 else x)


In [5]:
train_df = merged_df.copy()

In [7]:
import re

# Fungsi untuk membersihkan data penulis
def clean_authors(authors):
    if pd.isna(authors):  # Jika nilai NaN, ganti dengan string kosong
        return ""
    # Ganti tanda ; menjadi koma
    authors = authors.replace(';', ',')
    # Ganti karakter selain alfabet, angka, koma, dan spasi jadi spasi
    authors = re.sub(r"[^a-zA-Z0-9,\s]", " ", authors)
    # Hapus spasi ganda
    authors = re.sub(r"\s+", " ", authors).strip()
    return authors

# Terapkan fungsi ke kolom authors
train_df['authors_paper'] = train_df['authors_paper'].apply(clean_authors)
train_df['authors_referenced'] = train_df['authors_referenced'].apply(clean_authors)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize title dengan TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# TF-IDF untuk judul dokumen utama
title_paper_tfidf = tfidf.fit_transform(train_df['title_paper'])

# TF-IDF untuk judul dokumen yang dirujuk
title_referenced_tfidf = tfidf.transform(train_df['title_referenced'])


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

# Pecah konsep menjadi daftar
train_df['concepts_paper'] = train_df['concepts_paper'].apply(lambda x: x.split(';'))
train_df['concepts_referenced'] = train_df['concepts_referenced'].apply(lambda x: x.split(';'))

# One-hot encoding untuk konsep
mlb = MultiLabelBinarizer()
concepts_paper_encoded = mlb.fit_transform(train_df['concepts_paper'])
concepts_referenced_encoded = mlb.transform(train_df['concepts_referenced'])

# Menampilkan jumlah konsep unik
print("Jumlah konsep unik:", len(mlb.classes_))


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [' 2019-20 coronavirus outbreak', ' 3D reconstruction', ' 3D ultrasound', ' ARM architecture', ' Ab initio', ' Ab initio quantum chemistry methods', ' Absolute (philosophy)', ' Abstract interpretation', ' Abstract syntax tree', ' Abstraction', ' Accelerometer', ' Accounting', ' Accuracy and precision', ' Acoustics', ' Acronym', ' Action learning', ' Action recognition', ' Action selection', ' Active appearance model', ' Active listening', ' Activities of daily living', ' Activity recognition', ' Adaptability', ' Adaptive sampling', ' Adaptive value', ' Adenocarcinoma', ' Adenoma', ' Adjacency list', ' Adversary', ' Adverse effect', ' Advertising', ' Aerodynamics', ' Aesthetics', ' Affine hull', ' Affine transformation', ' African american', ' Agency (philosophy)', ' Aggregate (composite)', ' Agile software development', ' Agronomy', ' Air quality index', ' Alanine', ' Algebraic n

Jumlah konsep unik: 1055


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Hitung kesamaan judul
title_similarity = cosine_similarity(title_paper_tfidf, title_referenced_tfidf)

# Hitung kesamaan konsep
concept_similarity = cosine_similarity(concepts_paper_encoded, concepts_referenced_encoded)

# Gabungkan fitur untuk dataset final
features = np.hstack([
    title_similarity,
    concept_similarity,
    train_df[['year_difference']].values
])

# Label untuk prediksi
labels = train_df['is_referenced']


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# Example: Simulate dataset `train_df`
# Ensure you replace this with your actual dataset
# train_df = pd.read_csv('merged_data.csv')

# Define `features` and `labels`
features = train_df.drop(columns=['is_referenced'])  # Drop target column
labels = train_df['is_referenced']  # Target column

# Handle categorical or text columns (e.g., "concepts" or "authors") - You must preprocess them here
# For simplicity, this assumes features are already numeric

# Undersample the majority class to handle class imbalance
data = pd.concat([features, labels], axis=1)
majority_class = data[data['is_referenced'] == 0]
minority_class = data[data['is_referenced'] == 1]

# Undersample the majority class
majority_downsampled = resample(majority_class,
                                replace=False,  # Sample without replacement
                                n_samples=len(minority_class),  # Match minority class size
                                random_state=42)

# Combine the undersampled data
balanced_data = pd.concat([majority_downsampled, minority_class])

# Redefine `features` and `labels` after undersampling
features = balanced_data.drop(columns=['is_referenced'])
labels = balanced_data['is_referenced']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Scale features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Akurasi:", accuracy_score(y_test, y_pred))


NameError: name 'train_df' is not defined

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Use Random Forest for prediction
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the results
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


NameError: name 'features' is not defined

In [11]:
train_df.head()

,paper,referenced_paper,is_referenced,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_referenced,title_referenced,cited_by_count_referenced,type_referenced,authors_referenced,concepts_referenced,year_difference
0,p2128,p3728,0,p2128,a survey of data augmentation approaches for nlp,357,article,"steven y feng, varun gangal, jason wei, sarath...","[Computer science, Popularity, Artificial in...",p3728,optimization methods for large scale machine l...,2492,article,"l on bottou, frank e curtis, jorge nocedal","[Computer science, Machine learning, Artific...",3
1,p0389,p3811,0,p0389,residual algorithms reinforcement learning wit...,981,book-chapter,leemon c baird,"[Residual, Algorithm, Reinforcement learning...",p3811,filter pruning via geometric median for deep c...,1078,article,"yang he, ping liu, ziwei wang, zhilan hu, yi yang","[FLOPS, Computer science, Convolutional neur...",-1
2,p1298,p3760,0,p1298,segnet a deep convolutional encoder decoder ar...,16255,article,"vijay badrinarayanan, a c kendall, roberto cip...","[Computer science, Artificial intelligence, ...",p3760,integrative methods for analyzing big data in ...,182,review,"vladimir gligorijevi , no l malod dognin, nata...","[Big data, Data science, Precision medicine,...",2
3,p0211,p1808,0,p0211,deeplab semantic image segmentation with deep ...,18641,article,"liang chieh chen, george papandreou, iasonas k...","[Conditional random field, Artificial intelli...",p1808,building watson an overview of the deepqa project,1479,article,"david ferrucci, eric w brown, jennifer chu car...","[Watson, Champion, IBM, Computer science, ...",7
4,p0843,p2964,0,p0843,particle swarm optimization algorithm and its ...,799,review,ahmed g gad,"[Particle swarm optimization, Swarm intellige...",p2964,linear least squares algorithms for temporal d...,645,article,"steven j bradtke, andrew g barto","[Recursive least squares filter, Algorithm, ...",26


In [39]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, matthews_corrcoef
from scipy import sparse
import re
import time

def clean_text(text):
    """Clean and standardize text data"""
    if pd.isna(text):
        return ""
    text = re.sub(r"[^a-zA-Z0-9,\s]", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

def jaccard_similarity(list1, list2):
    """Calculate Jaccard similarity between two lists"""
    set1 = set(list1)
    set2 = set(list2)
    if not set1 or not set2:
        return 0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

def batch_cosine_sim(a, b, batch_size=50000):
    """Compute cosine similarity in batches to avoid memory issues"""
    sims = []
    for i in range(0, a.shape[0], batch_size):
        batch_a = a[i:i+batch_size]
        batch_b = b[i:i+batch_size]
        batch_sim = cosine_similarity(batch_a, batch_b)
        sims.append(np.diag(batch_sim))
    return np.concatenate(sims)

# Load and prepare data
print("Loading data...")
start_time = time.time()
train_df = pd.read_csv('/content/merged_data.csv')  # Replace with your actual data path

# Clean and preprocess data
print("Cleaning data...")
train_df['authors_paper'] = train_df['authors_paper'].apply(clean_text)
train_df['authors_referenced'] = train_df['authors_referenced'].apply(clean_text)
train_df['title_paper'] = train_df['title_paper'].apply(clean_text)
train_df['title_referenced'] = train_df['title_referenced'].apply(clean_text)

# Convert concepts to lists
train_df['concepts_paper'] = train_df['concepts_paper'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])
train_df['concepts_referenced'] = train_df['concepts_referenced'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])

# Calculate year difference
# train_df['year_difference'] = train_df['publication_year_paper'] - train_df['publication_year_referenced']

# Downsample majority class if needed
print("Balancing classes...")
pos_count = train_df['is_referenced'].sum()
neg_count = len(train_df) - pos_count
sample_size = min(150000, pos_count * 2)  # Adjust based on your memory

if pos_count < neg_count:
    pos_samples = train_df[train_df['is_referenced'] == 1]
    neg_samples = train_df[train_df['is_referenced'] == 0].sample(
        n=min(sample_size, neg_count), random_state=42)
    train_df = pd.concat([pos_samples, neg_samples]).sample(frac=1, random_state=42)

# Feature Engineering
print("Creating features...")

# 1. Title similarity using HashingVectorizer (memory efficient)
hash_vectorizer = HashingVectorizer(n_features=2**18, stop_words='english')
title_paper = hash_vectorizer.transform(train_df['title_paper'])
title_ref = hash_vectorizer.transform(train_df['title_referenced'])
title_sim = batch_cosine_sim(title_paper, title_ref)

# 2. Concept similarity
train_df['concept_sim'] = train_df.apply(
    lambda x: jaccard_similarity(x['concepts_paper'], x['concepts_referenced']), axis=1)

# 3. Author overlap
train_df['author_overlap'] = train_df.apply(
    lambda x: jaccard_similarity(
        x['authors_paper'].split(','),
        x['authors_referenced'].split(',')
    ), axis=1)

# 4. Citation features (handle missing values)
train_df['citation_ratio'] = (train_df['cited_by_count_paper'] + 1) / (train_df['cited_by_count_referenced'] + 1)

# Combine all features
features = np.column_stack([
    title_sim,
    train_df['concept_sim'],
    train_df['author_overlap'],
    train_df['year_difference'],
    train_df['citation_ratio'],
    np.log1p(train_df['cited_by_count_paper']),
    np.log1p(train_df['cited_by_count_referenced'])
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    features, train_df['is_referenced'], test_size=0.2, random_state=42)

# Train model
print("Training model...")
model = HistGradientBoostingClassifier(
    max_iter=200,
    learning_rate=0.05,
    max_depth=6,
    min_samples_leaf=20,
    l2_regularization=1.0,
    early_stopping=True,
    scoring='roc_auc',
    random_state=42,
    verbose=1
)

model.fit(X_train, y_train)

# Evaluate
print("Evaluating model...")
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nMatthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

print(f"\nTotal execution time: {(time.time() - start_time)/60:.2f} minutes")

Loading data...
Cleaning data...
Balancing classes...
Creating features...
Training model...
Binning 0.001 GB of training data: 0.021 s
Binning 0.000 GB of validation data: 0.001 s
Fitting gradient boosted rounds:
Fit 39 trees in 0.402 s, (1209 total leaves)
Time spent computing histograms: 0.030s
Time spent finding best splits:  0.016s
Time spent applying splits:      0.027s
Time spent predicting:           0.002s
Evaluating model...

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1736
           1       0.78      0.64      0.70       840

    accuracy                           0.82      2576
   macro avg       0.81      0.77      0.79      2576
weighted avg       0.82      0.82      0.82      2576


Matthews Correlation Coefficient: 0.5822162324778627

Total execution time: 0.37 minutes


In [40]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Tambahkan Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Latih masing-masing model
print("Training Random Forest...")
rf_model.fit(X_train, y_train)

print("Training HistGradientBoostingClassifier...")
hgb_model = HistGradientBoostingClassifier(
    max_iter=200,
    learning_rate=0.05,
    max_depth=6,
    min_samples_leaf=20,
    l2_regularization=1.0,
    early_stopping=True,
    scoring='roc_auc',
    random_state=42,
    verbose=1
)
hgb_model.fit(X_train, y_train)

# Ensemble Voting Classifier (Soft voting menggunakan probabilitas)
ensemble = VotingClassifier(
    estimators=[
        ('hgb', hgb_model),
        ('rf', rf_model)
    ],
    voting='soft',  # menggunakan probabilitas prediksi
    weights=[2, 1],  # opsional: memberi bobot lebih pada model tertentu
    n_jobs=-1
)

print("Training Ensemble Model (Voting Classifier)...")
ensemble.fit(X_train, y_train)

# Evaluasi semua model
from sklearn.metrics import classification_report, matthews_corrcoef

def evaluate_model(name, model, X_test, y_test):
    print(f"\nEvaluating: {name}")
    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Evaluasi per model
evaluate_model("Random Forest", rf_model, X_test, y_test)
evaluate_model("HistGradientBoosting", hgb_model, X_test, y_test)
evaluate_model("Ensemble (Voting Classifier)", ensemble, X_test, y_test)


Training Random Forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s


Training HistGradientBoostingClassifier...
Binning 0.001 GB of training data: 0.012 s
Binning 0.000 GB of validation data: 

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.1s finished


0.001 s
Fitting gradient boosted rounds:
Fit 39 trees in 0.495 s, (1209 total leaves)
Time spent computing histograms: 0.043s
Time spent finding best splits:  0.025s
Time spent applying splits:      0.041s
Time spent predicting:           0.003s
Training Ensemble Model (Voting Classifier)...

Evaluating: Random Forest
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1736
           1       0.75      0.65      0.70       840

    accuracy                           0.82      2576
   macro avg       0.80      0.77      0.78      2576
weighted avg       0.81      0.82      0.81      2576

Matthews Correlation Coefficient: 0.5672436819844806

Evaluating: HistGradientBoosting
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1736
           1       0.78      0.64      0.70       840

    accuracy                           0.82      2576
  

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
import joblib

# Save the trained model to a file
model_filename = "hist_gradient_boosting_model.pkl"
joblib.dump(model, model_filename)
print(f"Model saved to {model_filename}")


In [11]:
# Load the model from the file
loaded_model = joblib.load("/content/hist_gradient_boosting_model.pkl")
print("Model loaded successfully")


Model loaded successfully


In [4]:
df = pd.read_csv("/content/metadata.csv", encoding='latin-1') # Try 'latin-1' encoding, or others like 'cp1252', 'iso-8859-1' if 'latin-1' fails.
                                                                # Experiment with different encodings until you find the one that works.

In [5]:
df.drop(columns=['publication_date'], inplace=True)

In [6]:
import re

def clean_authors(authors):
    if pd.isna(authors):
        return ""
    # Ganti tanda ; menjadi koma
    authors = authors.replace(';', ',')
    # Ganti karakter non-alfabet/non-koma (selain koma dan spasi) jadi spasi
    authors = re.sub(r"[^a-zA-Z0-9,\s]", " ", authors)
    # Hapus spasi ganda
    authors = re.sub(r"\s+", " ", authors).strip()
    return authors


In [6]:
df['authors'] = df['authors'].apply(clean_authors)

In [12]:
df.head()

,paper,title,publication_year,cited_by_count,type,authors,concepts
0,p0000,Machine Learning in Medicine,2015,2662,review,Rahul C Deo,Medicine; Medical physics; Medical education; ...
1,p0001,A literature survey of benchmark functions for...,2013,1138,article,"Momin Jamil, Xin She Yang",Benchmark (surveying); Set (abstract data type...
2,p0002,Abnormal event detection in videos using gener...,2017,486,article,"Mahdyar Ravanbakhsh, Moin Nabi, Enver Sanginet...",Abnormality; Computer science; Artificial inte...
3,p0003,On Using Very Large Target Vocabulary for Neur...,2015,916,article,"S bastien Jean, Kyunghyun Cho, Roland Memisevi...",Machine translation; Computer science; Vocabul...
4,p0004,Gaussian Process Dynamical Models for Human Mo...,2007,1016,article,"Jonathan M Wang, David J Fleet, Aaron Hertzmann",Gaussian process; Artificial intelligence; Lat...


In [8]:
test = pd.read_csv('/content/data/test.csv')

In [9]:
papers_metadata = df.copy()

In [10]:
# Merge paper features
test = test.merge(papers_metadata, left_on='paper', right_on='paper_id', how='left')
test = test.merge(papers_metadata, left_on='referenced_paper', right_on='paper_id',
                             how='left', suffixes=('_paper', '_ref'))

In [23]:
test.head()

,id,paper,referenced_paper,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_ref,title_ref,cited_by_count_ref,type_ref,authors_ref,concepts_ref,year_difference
0,0,p0913,p3488,p0913,Profiles in self-regulated learning in the onl...,401,article,"Lucy Barnard Brak, Valerie Osland Paton, Willi...",Self-regulated learning; Likert scale; Psychol...,p3488,Three generations of distance education pedagogy,933,article,"Terry Anderson, Jon Dron",Distance education; Pedagogy; Community of inq...,-1
1,1,p2971,p4337,p2971,Feature engineering and symbolic regression me...,74,article,"Harsha Vaddireddy, Adil Rasheed, Anne Staples,...",Physics; Statistical physics; Feature (linguis...,p4337,Situated Cognition and the Culture of Learning,12629,article,"John Seely Brown, Allan Collins, Paul Duguid",Situated; Situated cognition; Cognitive appren...,31
2,2,p2237,p1610,p2237,Bayesian Inference of Sampled Ancestor Trees f...,349,article,"Alexandra Gavryushkina, David Welch, Tanja Sta...",Ancestor; Phylogenetic tree; Inference; Markov...,p1610,The Millennium Development Goals Report,3729,dataset,BRILL,Millennium Development Goals; Geography; Envir...,-1
3,3,p2876,p3212,p2876,Towards implementation of AI in New Zealand na...,21,article,"Li Xie, Song Yang, David Squirrell, Ehsan Vaghefi",Computer science; Convolutional neural network...,p3212,Branching and interacting particle systems app...,274,book-chapter,"Pierre Del Moral, Laurent Miclo",Mathematics; Feynman diagram; Particle system;...,20
4,4,p2939,p1901,p2939,SemEval-2016 Task 4: Sentiment Analysis in Twi...,592,article,"Preslav Nakov, Alan Ritter, Sara Rosenthal, Fa...",SemEval; Sentiment analysis; Computer science;...,p1901,Decentralized Reinforcement Learning Control o...,51,article,"Lucian Bu oniu, Bart De Schutter, Robert Babu ka",Reinforcement learning; Variety (cybernetics);...,10


In [11]:
test['year_difference'] = test['publication_year_paper'] - test['publication_year_ref']

In [12]:
test.drop(columns=['publication_year_paper', 'publication_year_ref'], inplace=True)

In [13]:
test['year_difference'] = test['year_difference'].apply(lambda x: -1 if x < 0 else x)


In [14]:
new_df = test.copy()

In [15]:
import re

# Clean text for authors, title, etc.
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"[^a-zA-Z0-9,\s]", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

# Clean authors and titles
new_df['authors_paper'] = new_df['authors_paper'].apply(clean_text)
new_df['authors_ref'] = new_df['authors_ref'].apply(clean_text)
new_df['title_paper'] = new_df['title_paper'].apply(clean_text)
new_df['title_ref'] = new_df['title_ref'].apply(clean_text)

# Convert concepts to lists
new_df['concepts_paper'] = new_df['concepts_paper'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])
new_df['concepts_ref'] = new_df['concepts_ref'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])

# # Calculate year difference
# new_df['year_difference'] = new_df['publication_year_paper'] - new_df['publication_year_referenced']
# new_df['year_difference'] = new_df['year_difference'].apply(lambda x: -1 if x < 0 else x)


In [17]:
new_df.to_csv('new_df.csv', index=False)

In [16]:
new_df.head()

,id,paper,referenced_paper,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_ref,title_ref,cited_by_count_ref,type_ref,authors_ref,concepts_ref,year_difference
0,0,p0913,p3488,p0913,profiles in self regulated learning in the onl...,401,article,lucy barnard brak valerie osland paton william...,"[Self-regulated learning, Likert scale, Psyc...",p3488,three generations of distance education pedagogy,933,article,terry anderson jon dron,"[Distance education, Pedagogy, Community of ...",-1
1,1,p2971,p4337,p2971,feature engineering and symbolic regression me...,74,article,harsha vaddireddy adil rasheed anne staples om...,"[Physics, Statistical physics, Feature (ling...",p4337,situated cognition and the culture of learning,12629,article,john seely brown allan collins paul duguid,"[Situated, Situated cognition, Cognitive app...",31
2,2,p2237,p1610,p2237,bayesian inference of sampled ancestor trees f...,349,article,alexandra gavryushkina david welch tanja stadl...,"[Ancestor, Phylogenetic tree, Inference, Ma...",p1610,the millennium development goals report,3729,dataset,brill,"[Millennium Development Goals, Geography, En...",-1
3,3,p2876,p3212,p2876,towards implementation of ai in new zealand na...,21,article,li xie song yang david squirrell ehsan vaghefi,"[Computer science, Convolutional neural netwo...",p3212,branching and interacting particle systems app...,274,book-chapter,pierre del moral laurent miclo,"[Mathematics, Feynman diagram, Particle syst...",20
4,4,p2939,p1901,p2939,semeval 2016 task 4 sentiment analysis in twitter,592,article,preslav nakov alan ritter sara rosenthal fabri...,"[SemEval, Sentiment analysis, Computer scien...",p1901,decentralized reinforcement learning control o...,51,article,lucian bu oniu bart de schutter robert babu ka,"[Reinforcement learning, Variety (cybernetics...",10


In [2]:
import pandas as pd
new_df = pd.read_csv('/content/new_df.csv')

In [2]:
new_df.shape

(336021, 16)

In [37]:
new_df.tail()

,id,paper,referenced_paper,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_ref,title_ref,cited_by_count_ref,type_ref,authors_ref,concepts_ref,year_difference
336016,336016,p0778,p2030,p0778,active learning via transductive experimental ...,348,article,kai yu jinbo bi volker tresp,"['Computer science', ' Machine learning', ' Ar...",p2030,patiency is not a virtue the design of intelli...,176,article,joanna j bryson,"['Autonomy', ' Epistemology', ' Sociology', ' ...",-1
336017,336017,p3656,p2661,p3656,key concepts in ai safety robustness and adver...,5,report,tim g j rudner helen toner,"['Robustness (evolution)', ' Adversarial syste...",p2661,measuring knowledge of natural selection a com...,223,article,ross h nehm irvin sam schonfeld,"['Selection (genetic algorithm)', ' Test (biol...",13
336018,336018,p3088,p2517,p3088,time series forecasting of covid 19 transmissi...,962,article,vinay kumar reddy chimmula lei zhang,"['Coronavirus disease 2019 (COVID-19)', ' Outb...",p2517,reinforcement learning trees,9,article,ruoqing zhu donglin zeng michael r kosorok,"['Reinforcement', ' Reinforcement learning', '...",5
336019,336019,p1789,p2958,p1789,the knowledge within methods for data free mod...,87,article,matan haroush itay hubara elad hoffer daniel s...,"['Computer science', ' Normalization (sociolog...",p2958,encyclopedia of artificial intelligence,1379,book,stuart c shapiro,"['Encyclopedia', ' Cognitive science', ' Compu...",11
336020,336020,p2983,p2109,p2983,pct point cloud transformer,1292,article,meng hao guo jun xiong cai zheng ning liu tai ...,"['Point cloud', ' Computer science', ' Segment...",p2109,doing things twice or differently strategies t...,1,preprint,gopal sarma,"['Open science', ' Data science', ' Value (mat...",4


In [3]:
import pandas as pd

# Number of subsets
num_subsets = 20

# Calculate the size of each subset
subset_size = len(new_df) // num_subsets

# Loop to create and save each subset
for i in range(num_subsets):
    start_idx = i * subset_size
    # For the last subset, include the remaining rows
    end_idx = (i + 1) * subset_size if i != num_subsets - 1 else len(new_df)

    # Create the subset
    subset = new_df.iloc[start_idx:end_idx]

    # Save the subset to a new CSV file
    subset.to_csv(f'subset_{i + 1}.csv', index=False)
    print(f"Subset {i + 1} saved with shape: {subset.shape}")


Subset 1 saved with shape: (16801, 16)
Subset 2 saved with shape: (16801, 16)
Subset 3 saved with shape: (16801, 16)
Subset 4 saved with shape: (16801, 16)
Subset 5 saved with shape: (16801, 16)
Subset 6 saved with shape: (16801, 16)
Subset 7 saved with shape: (16801, 16)
Subset 8 saved with shape: (16801, 16)
Subset 9 saved with shape: (16801, 16)
Subset 10 saved with shape: (16801, 16)
Subset 11 saved with shape: (16801, 16)
Subset 12 saved with shape: (16801, 16)
Subset 13 saved with shape: (16801, 16)
Subset 14 saved with shape: (16801, 16)
Subset 15 saved with shape: (16801, 16)
Subset 16 saved with shape: (16801, 16)
Subset 17 saved with shape: (16801, 16)
Subset 18 saved with shape: (16801, 16)
Subset 19 saved with shape: (16801, 16)
Subset 20 saved with shape: (16802, 16)


In [25]:
# subset_1 = pd.read_csv('/content/subset_1.csv')
# subset_2 = pd.read_csv('/content/subset_2.csv')
# subset_3 = pd.read_csv('/content/subset_3.csv')
# subset_4 = pd.read_csv('/content/subset_4.csv')
# subset_5 = pd.read_csv('/content/subset_5.csv')
# subset_6 = pd.read_csv('/content/subset_6.csv')
# subset_7 = pd.read_csv('/content/subset_7.csv')
# subset_8 = pd.read_csv('/content/subset_8.csv')
# subset_9 = pd.read_csv('/content/subset_9.csv')
# subset_10 = pd.read_csv('/content/subset_10.csv')
subset_11 = pd.read_csv('/content/subset_11.csv')
subset_12 = pd.read_csv('/content/subset_12.csv')
subset_13 = pd.read_csv('/content/subset_13.csv')
subset_14 = pd.read_csv('/content/subset_14.csv')
subset_15 = pd.read_csv('/content/subset_15.csv')
subset_16 = pd.read_csv('/content/subset_16.csv')
subset_17 = pd.read_csv('/content/subset_17.csv')
subset_18 = pd.read_csv('/content/subset_18.csv')
subset_19 = pd.read_csv('/content/subset_19.csv')
subset_20 = pd.read_csv('/content/subset_20.csv')

In [41]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# HashingVectorizer for titles
hash_vectorizer = HashingVectorizer(n_features=2**18, stop_words='english')
title_paper = hash_vectorizer.transform(subset_20['title_paper'])
title_ref = hash_vectorizer.transform(subset_20['title_ref'])

# Title similarity
def batch_cosine_sim(a, b, batch_size=50000):
    sims = []
    for i in range(0, a.shape[0], batch_size):
        batch_a = a[i:i+batch_size]
        batch_b = b[i:i+batch_size]
        batch_sim = cosine_similarity(batch_a, batch_b)
        sims.append(np.diag(batch_sim))
    return np.concatenate(sims)

title_sim = batch_cosine_sim(title_paper, title_ref)

# Concept similarity using Jaccard similarity
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    if not set1 or not set2:
        return 0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

subset_20['concept_sim'] = subset_20.apply(
    lambda x: jaccard_similarity(x['concepts_paper'], x['concepts_ref']), axis=1)

# Author overlap using Jaccard similarity
subset_20['author_overlap'] = subset_20.apply(
    lambda x: jaccard_similarity(
        x['authors_paper'].split(','),
        x['authors_ref'].split(',')
    ), axis=1)

# Citation features
subset_20['citation_ratio'] = (subset_20['cited_by_count_paper'] + 1) / (subset_20['cited_by_count_ref'] + 1)

# Combine features into a feature matrix
features = np.column_stack([
    title_sim,
    subset_20['concept_sim'],
    subset_20['author_overlap'],
    subset_20['year_difference'],
    subset_20['citation_ratio'],
    np.log1p(subset_20['cited_by_count_paper']),
    np.log1p(subset_20['cited_by_count_ref'])
])

# Predict labels for new data
y_pred = ensemble.predict(features)
y_proba = ensemble.predict_proba(features)[:, 1]

# Add predictions to the new dataframe
subset_20['is_referenced_pred'] = y_pred
subset_20['is_referenced_proba'] = y_proba

# Save the results to a new file
subset_20.to_csv('/content/subset20_predict.csv', index=False)
print("Predictions saved!")


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


Predictions saved!


In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import os

# Fungsi pembersih teks
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"[^a-zA-Z0-9,\s]", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

# Jaccard Similarity
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

# Cosine similarity in batch
def batch_cosine_sim(a, b, batch_size=50000):
    sims = []
    for i in range(0, a.shape[0], batch_size):
        batch_a = a[i:i+batch_size]
        batch_b = b[i:i+batch_size]
        batch_sim = cosine_similarity(batch_a, batch_b)
        sims.append(np.diag(batch_sim))
    return np.concatenate(sims)

# Hashing Vectorizer (invariant, define once)
hash_vectorizer = HashingVectorizer(n_features=2**18, stop_words='english')

# Loop over subset_1 to subset_20
for i in range(1, 21):
    print(f"Processing subset_{i}...")

    # Load data
    path = f'/content/subset_{i}.csv'
    if not os.path.exists(path):
        print(f"File {path} not found, skipping...")
        continue

    df = pd.read_csv(path)

    # Preprocessing
    df['title_paper'] = df['title_paper'].apply(clean_text)
    df['title_ref'] = df['title_ref'].apply(clean_text)
    df['authors_paper'] = df['authors_paper'].apply(clean_text)
    df['authors_ref'] = df['authors_ref'].apply(clean_text)

    df['concepts_paper'] = df['concepts_paper'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
    df['concepts_ref'] = df['concepts_ref'].apply(lambda x: x.split(';') if isinstance(x, str) else [])

    # Feature Engineering
    title_paper_vec = hash_vectorizer.transform(df['title_paper'])
    title_ref_vec = hash_vectorizer.transform(df['title_ref'])
    title_sim = batch_cosine_sim(title_paper_vec, title_ref_vec)

    df['concept_sim'] = df.apply(lambda x: jaccard_similarity(x['concepts_paper'], x['concepts_ref']), axis=1)
    df['author_overlap'] = df.apply(
        lambda x: jaccard_similarity(x['authors_paper'].split(','), x['authors_ref'].split(',')), axis=1)
    df['citation_ratio'] = (df['cited_by_count_paper'] + 1) / (df['cited_by_count_ref'] + 1)

    # Feature Matrix
    features = np.column_stack([
        title_sim,
        df['concept_sim'],
        df['author_overlap'],
        df['year_difference'],
        df['citation_ratio'],
        np.log1p(df['cited_by_count_paper']),
        np.log1p(df['cited_by_count_ref'])
    ])

    # Prediction
    y_pred = ensemble.predict(features)
    y_proba = ensemble.predict_proba(features)[:, 1]

    df['is_referenced_pred'] = y_pred
    df['is_referenced_proba'] = y_proba

    # Save result
    df.to_csv(f'/content/subset_{i}_predict.csv', index=False)
    print(f"Saved: subset_{i}_predict.csv")

print("All subsets processed.")


Processing subset_1...


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


Saved: subset_1_predict.csv
Processing subset_2...


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


Saved: subset_2_predict.csv
Processing subset_3...


...

Saved: subset_18_predict.csv
Processing subset_19...


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


Saved: subset_19_predict.csv
Processing subset_20...


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


Saved: subset_20_predict.csv
All subsets processed.


In [44]:
import pandas as pd
import os

# List untuk menampung semua data
all_results = []

# Loop dari subset_1 hingga subset_20
for i in range(1, 21):
    file_path = f'/content/subset_{i}_predict.csv'

    if os.path.exists(file_path):
        df = pd.read_csv(file_path, usecols=['id', 'is_referenced_pred'])
        all_results.append(df)
        print(f"Loaded: subset_{i}_predict.csv")
    else:
        print(f"File not found: subset_{i}_predict.csv (skipping)")

# Gabungkan semua DataFrame
combined_df = pd.concat(all_results, ignore_index=True)

# Simpan ke file CSV
combined_df.to_csv('/content/all_predictions.csv', index=False)
print("All predictions combined and saved to all_predictions.csv.")


Loaded: subset_1_predict.csv
Loaded: subset_2_predict.csv
Loaded: subset_3_predict.csv
Loaded: subset_4_predict.csv
Loaded: subset_5_predict.csv
Loaded: subset_6_predict.csv
Loaded: subset_7_predict.csv
Loaded: subset_8_predict.csv
Loaded: subset_9_predict.csv
Loaded: subset_10_predict.csv
Loaded: subset_11_predict.csv
Loaded: subset_12_predict.csv
Loaded: subset_13_predict.csv
Loaded: subset_14_predict.csv
Loaded: subset_15_predict.csv
Loaded: subset_16_predict.csv
Loaded: subset_17_predict.csv
Loaded: subset_18_predict.csv
Loaded: subset_19_predict.csv
Loaded: subset_20_predict.csv
All predictions combined and saved to all_predictions.csv.


In [35]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# HashingVectorizer for titles
hash_vectorizer = HashingVectorizer(n_features=2**18, stop_words='english')
title_paper = hash_vectorizer.transform(subset_19['title_paper'])
title_ref = hash_vectorizer.transform(subset_19['title_ref'])

# Title similarity
def batch_cosine_sim(a, b, batch_size=50000):
    sims = []
    for i in range(0, a.shape[0], batch_size):
        batch_a = a[i:i+batch_size]
        batch_b = b[i:i+batch_size]
        batch_sim = cosine_similarity(batch_a, batch_b)
        sims.append(np.diag(batch_sim))
    return np.concatenate(sims)

title_sim = batch_cosine_sim(title_paper, title_ref)

# Concept similarity using Jaccard similarity
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    if not set1 or not set2:
        return 0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

subset_19['concept_sim'] = subset_19.apply(
    lambda x: jaccard_similarity(x['concepts_paper'], x['concepts_ref']), axis=1)

# Author overlap using Jaccard similarity
subset_19['author_overlap'] = subset_19.apply(
    lambda x: jaccard_similarity(
        x['authors_paper'].split(','),
        x['authors_ref'].split(',')
    ), axis=1)

# Citation features
subset_19['citation_ratio'] = (subset_19['cited_by_count_paper'] + 1) / (subset_19['cited_by_count_ref'] + 1)

# Combine features into a feature matrix
features = np.column_stack([
    title_sim,
    subset_19['concept_sim'],
    subset_19['author_overlap'],
    subset_19['year_difference'],
    subset_19['citation_ratio'],
    np.log1p(subset_19['cited_by_count_paper']),
    np.log1p(subset_19['cited_by_count_ref'])
])

# Predict labels for new data
y_pred = loaded_model.predict(features)
y_proba = loaded_model.predict_proba(features)[:, 1]

# Add predictions to the new dataframe
subset_19['is_referenced_pred'] = y_pred
subset_19['is_referenced_proba'] = y_proba

# Save the results to a new file
subset_19.to_csv('/content/subset19_predict.csv', index=False)
print("Predictions saved!")


Predictions saved!


In [16]:
# Predict labels for new data
y_pred = loaded_model.predict(features)
y_proba = loaded_model.predict_proba(features)[:, 1]

# Add predictions to the new dataframe
subset_2['is_referenced_pred'] = y_pred
subset_2['is_referenced_proba'] = y_proba

# Save the results to a new file
subset_2.to_csv('/content/subset2_predict.csv', index=False)
print("Predictions saved!")


Predictions saved!


In [28]:
new_df.head()

,id,paper,referenced_paper,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_ref,title_ref,cited_by_count_ref,type_ref,authors_ref,concepts_ref,year_difference
0,0,p0913,p3488,p0913,profiles in self regulated learning in the onl...,401,article,"lucy barnard brak, valerie osland paton, willi...","[Self-regulated learning, Likert scale, Psyc...",p3488,three generations of distance education pedagogy,933,article,"terry anderson, jon dron","[Distance education, Pedagogy, Community of ...",-1
1,1,p2971,p4337,p2971,feature engineering and symbolic regression me...,74,article,"harsha vaddireddy, adil rasheed, anne staples,...","[Physics, Statistical physics, Feature (ling...",p4337,situated cognition and the culture of learning,12629,article,"john seely brown, allan collins, paul duguid","[Situated, Situated cognition, Cognitive app...",31
2,2,p2237,p1610,p2237,bayesian inference of sampled ancestor trees f...,349,article,"alexandra gavryushkina, david welch, tanja sta...","[Ancestor, Phylogenetic tree, Inference, Ma...",p1610,the millennium development goals report,3729,dataset,brill,"[Millennium Development Goals, Geography, En...",-1
3,3,p2876,p3212,p2876,towards implementation of ai in new zealand na...,21,article,"li xie, song yang, david squirrell, ehsan vaghefi","[Computer science, Convolutional neural netwo...",p3212,branching and interacting particle systems app...,274,book-chapter,"pierre del moral, laurent miclo","[Mathematics, Feynman diagram, Particle syst...",20
4,4,p2939,p1901,p2939,semeval 2016 task 4 sentiment analysis in twitter,592,article,"preslav nakov, alan ritter, sara rosenthal, fa...","[SemEval, Sentiment analysis, Computer scien...",p1901,decentralized reinforcement learning control o...,51,article,"lucian bu oniu, bart de schutter, robert babu ka","[Reinforcement learning, Variety (cybernetics...",10


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from scipy import sparse
import re
import time

def clean_text(text):
    """Clean and standardize text data"""
    if pd.isna(text):
        return ""
    text = re.sub(r"[^a-zA-Z0-9,\s]", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

def jaccard_similarity(list1, list2):
    """Calculate Jaccard similarity between two lists"""
    set1 = set(list1)
    set2 = set(list2)
    if not set1 or not set2:
        return 0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

def batch_cosine_sim(a, b, batch_size=50000):
    """Compute cosine similarity in batches to avoid memory issues"""
    sims = []
    for i in range(0, a.shape[0], batch_size):
        batch_a = a[i:i+batch_size]
        batch_b = b[i:i+batch_size]
        batch_sim = cosine_similarity(batch_a, batch_b)
        sims.append(np.diag(batch_sim))
    return np.concatenate(sims)

# Load and prepare data
print("Loading data...")
start_time = time.time()
train_df = pd.read_csv('/content/merged_data.csv')  # Replace with your actual data path

# Clean and preprocess data
print("Cleaning data...")
train_df['authors_paper'] = train_df['authors_paper'].apply(clean_text)
train_df['authors_referenced'] = train_df['authors_referenced'].apply(clean_text)
train_df['title_paper'] = train_df['title_paper'].apply(clean_text)
train_df['title_referenced'] = train_df['title_referenced'].apply(clean_text)

# Convert concepts to lists
train_df['concepts_paper'] = train_df['concepts_paper'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])
train_df['concepts_referenced'] = train_df['concepts_referenced'].apply(
    lambda x: x.split(';') if isinstance(x, str) else [])

# Calculate year difference
# train_df['year_difference'] = train_df['publication_year_paper'] - train_df['publication_year_referenced']

# Enhanced Feature Engineering
print("Creating enhanced features...")

# 1. Title similarity using different methods
hash_vectorizer = HashingVectorizer(n_features=2**18, stop_words='english')
title_paper_hash = hash_vectorizer.transform(train_df['title_paper'])
title_ref_hash = hash_vectorizer.transform(train_df['title_referenced'])
title_sim_hash = batch_cosine_sim(title_paper_hash, title_ref_hash)

# TF-IDF for better quality (but more memory-intensive)
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
# Only transform a subset if memory is an issue
sample_size = min(len(train_df), 100000)
tfidf_sample = train_df.iloc[:sample_size]
title_paper_tfidf = tfidf.fit_transform(tfidf_sample['title_paper'])
title_ref_tfidf = tfidf.transform(tfidf_sample['title_referenced'])
title_sim_tfidf = batch_cosine_sim(title_paper_tfidf, title_ref_tfidf)

# For samples beyond our tfidf subset, use the hash similarity
if sample_size < len(train_df):
    title_sim_all = np.zeros(len(train_df))
    title_sim_all[:sample_size] = title_sim_tfidf
    title_sim_all[sample_size:] = title_sim_hash[sample_size:]
else:
    title_sim_all = title_sim_tfidf

# 2. Concept similarity
train_df['concept_sim'] = train_df.apply(
    lambda x: jaccard_similarity(x['concepts_paper'], x['concepts_referenced']), axis=1)

# 3. Author overlap with improved method
def author_similarity(authors1, authors2):
    # More sophisticated author matching that handles variations
    if not authors1 or not authors2:
        return 0

    authors1_list = [a.strip() for a in authors1.split(',') if a.strip()]
    authors2_list = [a.strip() for a in authors2.split(',') if a.strip()]

    # Direct overlap (exact matches)
    direct_overlap = jaccard_similarity(authors1_list, authors2_list)

    # Last name matching (for cases where first names might be abbreviated)
    last_names1 = [name.split()[-1] if len(name.split()) > 0 else name for name in authors1_list]
    last_names2 = [name.split()[-1] if len(name.split()) > 0 else name for name in authors2_list]
    lastname_overlap = jaccard_similarity(last_names1, last_names2)

    # Combined score
    return max(direct_overlap, lastname_overlap)

train_df['author_overlap'] = train_df.apply(
    lambda x: author_similarity(x['authors_paper'], x['authors_referenced']), axis=1)

# 4. Citation features with better normalization
train_df['citation_ratio'] = np.log1p(train_df['cited_by_count_paper'] + 1) / np.log1p(train_df['cited_by_count_referenced'] + 1)
train_df['citation_diff'] = np.log1p(train_df['cited_by_count_paper']) - np.log1p(train_df['cited_by_count_referenced'])

# 5. Age-adjusted citation impact
# Citations per year for referenced paper
train_df['years_since_pub_ref'] = np.maximum(1, 2024 - train_df['publication_year_referenced'])
train_df['citation_rate_ref'] = train_df['cited_by_count_referenced'] / train_df['years_since_pub_ref']

# Combine all features
print("Building feature matrix...")
features = np.column_stack([
    title_sim_all,
    train_df['concept_sim'],
    train_df['author_overlap'],
    train_df['year_difference'],
    train_df['citation_ratio'],
    train_df['citation_diff'],
    np.log1p(train_df['cited_by_count_paper']),
    np.log1p(train_df['cited_by_count_referenced']),
    train_df['citation_rate_ref'],
    train_df['years_since_pub_ref']
])

# Scale numerical features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Balanced sampling
print("Balancing classes...")
pos_samples = train_df[train_df['is_referenced'] == 1]
neg_samples = train_df[train_df['is_referenced'] == 0]

# More balanced approach - keep all positive samples and sample negative ones
pos_count = len(pos_samples)
neg_count = len(neg_samples)
sample_size = min(200000, max(pos_count * 3, neg_count))

if pos_count < neg_count:
    neg_samples = neg_samples.sample(n=min(sample_size, neg_count), random_state=42)
    balanced_df = pd.concat([pos_samples, neg_samples])
    # Get the corresponding indices and features
    balanced_indices = balanced_df.index
    balanced_features = features_scaled[balanced_indices]
    balanced_labels = train_df.loc[balanced_indices, 'is_referenced']
else:
    balanced_features = features_scaled
    balanced_labels = train_df['is_referenced']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    balanced_features, balanced_labels, test_size=0.2, random_state=42, stratify=balanced_labels)

# Create individual models for the ensemble
print("Creating models for ensemble...")

# Model 1: Gradient Boosting
gb_model = HistGradientBoostingClassifier(
    max_iter=300,
    learning_rate=0.05,
    max_depth=8,
    min_samples_leaf=15,
    l2_regularization=0.5,
    early_stopping=True,
    random_state=42,
    verbose=0
)

# Model 2: Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features='sqrt',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

# Model 3: XGBoost
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42,
    verbosity=0
)

# Model 4: Logistic Regression (for diversity)
lr_model = LogisticRegression(
    C=0.1,
    class_weight='balanced',
    max_iter=1000,
    n_jobs=-1,
    random_state=42,
    verbose=0
)

# Create the voting ensemble
ensemble = VotingClassifier(
    estimators=[
        ('gb', gb_model),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lr', lr_model)
    ],
    voting='soft',  # Use probability estimates for voting
    n_jobs=-1,
    verbose=0
)

# Optional: Grid search for best voting weights
param_grid = {
    'weights': [
        [1, 1, 1, 1],    # Equal weights
        [2, 1, 1, 1],    # Emphasize GB
        [1, 2, 1, 1],    # Emphasize RF
        [1, 1, 2, 1],    # Emphasize XGB
        [3, 2, 3, 1],    # Custom weights
    ]
}

# Use a subset for grid search if data is large
grid_size = min(len(X_train), 50000)
X_grid = X_train[:grid_size]
y_grid = y_train[:grid_size]

print("Finding optimal ensemble weights...")
grid_search = GridSearchCV(
    ensemble,
    param_grid,
    cv=3,
    scoring='matthews_corrcoef',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_grid, y_grid)

print(f"Best weights: {grid_search.best_params_}")

# Train the ensemble with the best weights
print("Training final ensemble model...")
best_ensemble = VotingClassifier(
    estimators=[
        ('gb', gb_model),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lr', lr_model)
    ],
    voting='soft',
    weights=grid_search.best_params_['weights'],
    n_jobs=-1,
    verbose=1
)

best_ensemble.fit(X_train, y_train)

# Evaluate individual models and ensemble
print("Evaluating models...")
models = {
    'Gradient Boosting': gb_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'Logistic Regression': lr_model,
    'Voting Ensemble': best_ensemble
}

results = {}
for name, model in models.items():
    if name != 'Voting Ensemble':  # Skip training individual models, they're already trained via the ensemble
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    results[name] = {
        'MCC': mcc,
        'F1': report['weighted avg']['f1-score'],
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'Accuracy': report['accuracy']
    }

    print(f"\n{name} Results:")
    print(f"MCC: {mcc:.4f}")
    print(classification_report(y_test, y_pred))

# Create results dataframe for comparison
results_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(results_df.sort_values('MCC', ascending=False))

# Calculate feature importances (from the gradient boosting model)
if hasattr(gb_model, 'feature_importances_'):
    feature_names = [
        'Title Similarity', 'Concept Similarity', 'Author Overlap',
        'Year Difference', 'Citation Ratio', 'Citation Difference',
        'Log Paper Citations', 'Log Reference Citations',
        'Citation Rate Reference', 'Years Since Publication'
    ]
    importances = gb_model.feature_importances_
    indices = np.argsort(importances)[::-1]

    print("\nFeature Importances:")
    for i in range(len(feature_names)):
        print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

print(f"\nTotal execution time: {(time.time() - start_time)/60:.2f} minutes")

Loading data...
Cleaning data...
Creating enhanced features...


In [35]:
train_df.head()

,paper,referenced_paper,is_referenced,paper_id_paper,title_paper,cited_by_count_paper,type_paper,authors_paper,concepts_paper,paper_id_referenced,title_referenced,cited_by_count_referenced,type_referenced,authors_referenced,concepts_referenced,year_difference
0,p2128,p3728,0,p2128,A Survey of Data Augmentation Approaches for NLP,357,article,"Steven Y Feng, Varun Gangal, Jason Wei, Sarath...","[Computer science, Popularity, Artificial in...",p3728,Optimization Methods for Large-Scale Machine L...,2492,article,"L on Bottou, Frank E Curtis, Jorge Nocedal","[Computer science, Machine learning, Artific...",3
1,p0389,p3811,0,p0389,Residual Algorithms: Reinforcement Learning wi...,981,book-chapter,Leemon C Baird,"[Residual, Algorithm, Reinforcement learning...",p3811,Filter Pruning via Geometric Median for Deep C...,1078,article,"Yang He, Ping Liu, Ziwei Wang, Zhilan Hu, Yi Yang","[FLOPS, Computer science, Convolutional neur...",-1
2,p1298,p3760,0,p1298,SegNet: A Deep Convolutional Encoder-Decoder A...,16255,article,"Vijay Badrinarayanan, A C Kendall, Roberto Cip...","[Computer science, Artificial intelligence, ...",p3760,Integrative methods for analyzing big data in ...,182,review,"Vladimir Gligorijevi , No l Malod Dognin, Nata...","[Big data, Data science, Precision medicine,...",2
3,p0211,p1808,0,p0211,DeepLab: Semantic Image Segmentation with Deep...,18641,article,"Liang Chieh Chen, George Papandreou, Iasonas K...","[Conditional random field, Artificial intelli...",p1808,Building Watson: An Overview of the DeepQA Pro...,1479,article,"David Ferrucci, Eric W Brown, Jennifer Chu Car...","[Watson, Champion, IBM, Computer science, ...",7
4,p0843,p2964,0,p0843,Particle Swarm Optimization Algorithm and Its ...,799,review,Ahmed G Gad,"[Particle swarm optimization, Swarm intellige...",p2964,Linear Least-Squares algorithms for temporal d...,645,article,"Steven J Bradtke, Andrew G Barto","[Recursive least squares filter, Algorithm, ...",26


In [28]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410691 entries, 0 to 410690
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   paper                      410691 non-null  object
 1   referenced_paper           410691 non-null  object
 2   is_referenced              410691 non-null  int64 
 3   paper_id_paper             410691 non-null  object
 4   title_paper                410691 non-null  object
 5   cited_by_count_paper       410691 non-null  int64 
 6   type_paper                 410691 non-null  object
 7   authors_paper              410691 non-null  object
 8   concepts_paper             410691 non-null  object
 9   paper_id_referenced        410691 non-null  object
 10  title_referenced           410691 non-null  object
 11  cited_by_count_referenced  410691 non-null  int64 
 12  type_referenced            410691 non-null  object
 13  authors_referenced         410691 non-null  

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410691 entries, 0 to 410690
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   paper             410691 non-null  object
 1   referenced_paper  410691 non-null  object
 2   is_referenced     410691 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 9.4+ MB
